In [1]:
%load_ext autoreload

In [2]:
%autoreload

In [3]:
import os
import json
from pipeline import complete_pipeline
from sklearn.pipeline import FeatureUnion

from feature_extractors import NDVIFeatureExtractor, ColorChannelsFeatureExtractor
from feature_extractors import LBPFeatureExtractor, GLCMFeatureExtractor
from feature_extractors import GCHFeatureExtractor

In [4]:
import sys
sys.path.append("../")
from config import RESULTS_PATH

# Initialise feature extractors

In [12]:
feature_extractors = [
    ('spectral_glcm_all', FeatureUnion([
        ('channels', ColorChannelsFeatureExtractor()),
        ('ndvi', NDVIFeatureExtractor()),
        ('glcm_all', GLCMFeatureExtractor(mode='all'))
    ]))
]

In [13]:
feature_extractors

[('spectral_glcm_all', FeatureUnion(n_jobs=None,
         transformer_list=[('channels', <feature_extractors.ColorChannelsFeatureExtractor object at 0x12ea75630>), ('ndvi', <feature_extractors.NDVIFeatureExtractor object at 0x12ea751d0>), ('glcm_all', <feature_extractors.GLCMFeatureExtractor object at 0x12ea754a8>)],
         transformer_weights=None))]

# evaluate performance

In [14]:
performances = {}
for feature_name, feature_extractor in feature_extractors:
    performance = complete_pipeline(feature_extractor)
    performances[feature_name] = performance

/Users/Samu/anaconda/envs/vision/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/Samu/anaconda/envs/vision/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[0.04590303 0.04316123 0.05490012 0.07593562 0.05548444 0.08788921
 0.04388603 0.07692315 0.10320305 0.06331204 0.05618627 0.06584061
 0.0429799  0.10732451 0.04040707 0.03666372]


In [15]:
performances

{'spectral_glcm_all': {'mean_f2': 0.8331971026996766,
  'per_class_f2': OrderedDict([('primary', 0.984411391065001),
               ('clear', 0.9567229648050288),
               ('agriculture', 0.7401724995853375),
               ('road', 0.5913561076604555),
               ('water', 0.5559593023255814),
               ('partly_cloudy', 0.7440174039158811),
               ('cultivation', 0.15191199580932427),
               ('habitation', 0.43258101851851855),
               ('haze', 0.504606365159129),
               ('cloudy', 0.6862745098039216),
               ('bare_ground', 0.006934812760055479),
               ('selective_logging', 0.0),
               ('artisinal_mine', 0.48657718120805366),
               ('blooming', 0.0),
               ('slash_burn', 0.0),
               ('conventional_mine', 0.0),
               ('blow_down', 0.0)]),
  'time_feature_extraction': 1153.3603389263153,
  'time_training': 215.76070308685303}}

In [16]:
for feature_name, performance in performances.items():
    file_path = os.path.join(RESULTS_PATH, f'{feature_name}.json')
    with open(file_path, 'w') as f:
        json.dump(performance, f, indent=4)